# Navigator 에이전트 구축

앞선 실습에서 우리는 `Browser-use` 에이전트가 자연어 명령 하나만으로 브라우저를 직접 조종하는 마법을 경험했습니다.
하지만 매번 브라우저를 새로 실행하고, 이전 대화를 전혀 기억하지 못한다면 실제 업무에서 쓰기엔 한계가 있겠죠?
이번 실습에서는 한 단계 더 나아가, **대화의 맥락을 기억하고 사용자와 자연스럽게 소통하는 'AI 웹 브라우징 어시스턴트'**, 즉 **Navigator 에이전트**를 직접 설계하고 구축합니다.
### 핵심 아이디어: Agent as Tool 패턴
이번 실습의 핵심은 **`Agent as Tool`** 패턴입니다. `Browser-use` 에이전트를 독립 실행 단위로 두는 것이 아니라, LangChain의 `@tool` 데코레이터로 감싸 **하나의 '도구(Tool)'로 전환**합니다.
그리고 이 도구를 더 똑똑한 상위 에이전트에게 쥐여주는 것입니다. 브라우저 조작이라는 '손발'과 대화·추론이라는 '머리'를 분리하는 것이 핵심입니다.

.env 파일을 로드하여 환경변수에 등록합니다. 노트북을 위한 실행 경로를 세팅합니다.

In [ ]:
import os, sys
from dotenv import load_dotenv

load_dotenv(override=True)
#load_dotenv("/mnt/d/agent_research/github-repos/home/AAWS_project/.env", override=True)

# PROJECT_ROOT를 .env에서 읽기 (없으면 현재 디렉토리)
project_root = os.getenv("PROJECT_ROOT", os.getcwd())

# 프로젝트 루트가 유효하지 않으면, 현재 위치에서 상위로 찾기
if not os.path.exists(os.path.join(project_root, "app")):
    # 상위 디렉토리 탐색
    current = os.getcwd()
    for _ in range(5):
        if os.path.exists(os.path.join(current, "app")):
            project_root = current
            break
        current = os.path.dirname(current)

# Working Directory 설정
os.chdir(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import app

print(f"✅ Project Root: {project_root}")
print(f"✅ Working Directory: {os.getcwd()}")

In [ ]:
import asyncio
from browser_use import Agent, Browser, ChatGoogle

import nest_asyncio

# 주피터 노트북 환경에서 비동기 루프 중복 실행 오류를 방지하기 위해 사용합니다.
nest_asyncio.apply()

# 가상 디스플레이(noVNC) 번호를 지정합니다.
# 이 설정이 있어야 GUI 브라우저가 가상 화면(:1)에 나타납니다.
os.environ["DISPLAY"] = ":1"

In [ ]:
os.environ["GOOGLE_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "develop_Navigator_agent"

In [ ]:
# ==========================================
# 1. 💡 런타임 공유 Context Data
# ==========================================
from dataclasses import dataclass
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain.agents import create_agent
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import InMemorySaver
from browser_use import Agent, ChatGoogle

@dataclass
class WebAssistantContext:
    # 사용자에 대한 기본 설정값 등을 넣을 수 있는 빈 컨텍스트로 둡니다.
    pass 

# ==========================================
# 2. 💡 범용 웹 탐색 도구 (Browser-use 기반)
# ==========================================
@tool
async def browse_web(instruction: str) -> str:
    """
    주어진 지시사항에 따라 웹 브라우저를 직접 조작하고 결과를 반환합니다.
    검색, 클릭, 텍스트 읽기, 웹사이트 분석 등 사용자가 요청한 웹 관련 작업을 수행할 때 사용하세요.
    
    Args:
        instruction: 브라우저가 수행해야 할 구체적인 행동 지시문 (예: '구글에서 2024년 올림픽 결과 검색해줘')
    """
    print(f"\n🌐 [Browser Tool] 행동 개시: {instruction}")
    
    bu_llm = ChatGoogle(model="gemini-flash-latest")
    
    # 전달받은 instruction을 그대로 Browser-use의 task로 전달합니다.
    agent = Agent(task=instruction, llm=bu_llm)
    history = await agent.run(max_steps=10)
    
    result_text = history.final_result()
    if not result_text:
        return "브라우저 조작을 시도했으나 명확한 결과를 얻지 못했습니다. 다른 명령으로 재시도해보세요."
    
    return result_text

# ==========================================
# 3. 💡 Web Assistant Agent (create_agent) 초기화
# ==========================================
# 멀티턴 대화를 위해 온도를 살짝 높여 자연스럽게 만듭니다 (0.1 -> 0.7)
model = init_chat_model("google_genai:gemini-flash-latest", temperature=0.7) 
store = InMemoryStore()
checkpointer = InMemorySaver()

SYSTEM_PROMPT = """
당신은 사용자와 대화하며 인터넷을 자유롭게 탐색하고 정보를 분석해주는 'AI 웹 브라우징 어시스턴트'입니다.

[역할 및 지침]
1. 당신은 `browse_web` 도구를 사용하여 스스로 웹 브라우저를 제어할 수 있습니다.
2. 사용자가 특정 사이트의 정보를 묻거나, 검색을 요청하거나, 웹 요약을 요구하면 적극적으로 도구를 사용하세요.
3. 도구를 사용할 때는 브라우저 에이전트(도구)가 명확하게 이해할 수 있도록 '구체적인 행동 명령(instruction)'을 작성하여 전달해야 합니다.
   - 나쁜 예: "네이버 뉴스 스크랩"
   - 좋은 예: "https://news.naver.com 에 접속해서 IT/과학 탭의 가장 위에 있는 기사 3개의 제목과 요약을 가져와"
4. 사용자와 자연스러운 멀티턴 대화를 이어나가며, 질문의 의도가 모호하다면 웹을 탐색하기 전에 되물어보세요.
5. 수집된 웹 탐색 결과를 단순히 전달하지 말고, 사용자의 문맥에 맞춰 요약하고 통찰력 있는 언어로 답변하세요.
"""

web_assistant_agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    context_schema=WebAssistantContext,
    tools=[browse_web],
    store=store,
    checkpointer=checkpointer)

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "my_browsing_session_1"}}
context = WebAssistantContext()

# 1번째 턴 (웹 탐색 유도)
query = "다나와 홈페이지에 접속해서 리뷰가 가장 많고 잘나가는 게이밍 마우스 상위 3개를 찾아봐."
print(f"👤 User: {query}")

response = await web_assistant_agent.ainvoke(
        {"messages": [HumanMessage(query)]},
        config=config,
        context=context
    )

print(f"\n🤖 Assistant: {response['messages'][-1].content}\n")
print("-" * 50)

제 지시대로 불명확한 지시의 경우 되묻습니다.

In [ ]:
# 2번째 턴
query = "가격과 이름을 알고 싶어."
print(f"👤 User: {query}")

response = await web_assistant_agent.ainvoke(
        {"messages": [HumanMessage(query)]},
        config=config,
        context=context
    )

print(f"\n🤖 Assistant: {response['messages'][-1].content}\n")
print("-" * 50)

In [ ]:
# 3번째 턴
query = "그 중 제일 싼 건 뭐야?"
print(f"👤 User: {query}")

response = await web_assistant_agent.ainvoke(
        {"messages": [HumanMessage(query)]},
        config=config,
        context=context
    )

print(f"\n🤖 Assistant: {response['messages'][-1].content}\n")
print("-" * 50)


In [ ]:
# 4번째 턴
query = "그 페이지에서 4번째로 인기있는 마우스의 이름은 뭔지 도구를 다시 쓰지 않고 답변할 수 있어?"
print(f"👤 User: {query}")

response = await web_assistant_agent.ainvoke(
        {"messages": [HumanMessage(query)]},
        config=config,
        context=context
    )

print(f"\n🤖 Assistant: {response['messages'][-1].content}\n")
print("-" * 50)

### 실습: 단절된 브라우저 탐색 맥락 연결하기

앞선 멀티턴 대화 실험(4번째 턴)에서 한 가지 문제가 발견되었습니다.
*"그 페이지에서 4번째로 인기있는 마우스의 이름은 뭐야?"* 라고 물었을 때, 에이전트가 보고 있던 페이지에서 찾지 않고 **처음부터 다시 다나와를 검색**했습니다.

Navigator 에이전트(상위)는 대화 내용을 기억하고 있지만, `browse_web` 도구가 호출될 때마다 **완전히 새로운 브라우저 세션이 생성**되기 때문에 실제 브라우저는 이전 탐색 맥락(내가 어느 페이지에 있었는지)을 잃어버리는 것입니다.

이 문제를 해결하는 두 가지 방법을 차례대로 실습해 봅시다.

---

#### 🛠️ 방법 1: 결과 텍스트에 방문 URL 함께 반환하기

가장 간단한 방법입니다. `browse_web` 도구가 작업 결과만 넘겨주는 것이 아니라, **마지막으로 도달한 URL**을 함께 적어서 반환합니다. 이렇게 하면 상위 에이전트가 다음 지시를 내릴 때 *"이 URL로 다시 접속해서..."* 라고 명령할 수 있습니다.


In [ ]:
# ==========================================
# 실습 1: URL을 반환하는 새로운 도구 정의
# ==========================================
from browser_use import Agent, ChatGoogle

@tool
async def browse_web_url_mode(instruction: str) -> str:
    # tool을 다시 구현하여 봅시다.


    return "구현 필요!"



# 새로운 도구를 장착한 에이전트로 업데이트
web_assistant_agent_v2 = create_agent(
    model=model, system_prompt=SYSTEM_PROMPT, context_schema=WebAssistantContext,
    tools=[browse_web_url_mode], store=store, checkpointer=checkpointer
)


아래 코드에서 테스트해봅시다.

In [ ]:
# 방법 1 테스트 
# URL을 받아서 어떻게 이어가는지 확인합니다.
config_v2 = {"configurable": {"thread_id": "test_session_url"}}

# 1턴: 검색 지시
await web_assistant_agent_v2.ainvoke(
    {"messages": [HumanMessage("네이버 뉴스 IT/과학 메인 페이지에 들어가서 첫 번째 기사 제목을 알려줘.")]},
    config=config_v2
)

print("-" * 50)
# 2턴: 맥락 질문 (불필요한 첫 페이지 검색 없이 전달받은 URL로 직접 가는지 확인)
response = await web_assistant_agent_v2.ainvoke(
    {"messages": [HumanMessage("그 페이지에서 두 번째 기사 제목은 뭐야?")]},
    config=config_v2
)
print(f"\n🤖 Assistant: {response['messages'][-1].content}")


---

#### 🛠️ 방법 2: 하나의 브라우저 세션 유지하기 (keep_alive=True)

URL을 넘겨주는 방법은 페이지를 새로고침해야 하므로 스크롤 위치나 동적 상태(로그인 정보)가 초기화된다는 한계가 있습니다. 
가장 완벽한 해결책은 **하나의 브라우저를 열어두고 끄지 않는 것(keep_alive=True)**입니다. 
`browse_web` 도구들이 하나의 브라우저 객체를 닫지 않고 공유해서 사용하도록 만들어 봅시다.


In [ ]:
from browser_use import Browser
# 💡 핵심: 세션을 유지하는 공유 브라우저 인스턴스를 하나만 생성합니다.
shared_browser = Browser(
    headless=False,
    disable_security=True,
    window_size={'width': 1280, 'height': 720},
    keep_alive=True  # 도구 호출이 끝나도 브라우저가 종료되지 않습니다.
)

@tool
async def browse_web_keep_alive(instruction: str) -> str:
    # tool을 다시 구현하여 봅시다.
    
    return "구현 필요!"

In [ ]:
# 방법 2 테스트 
# 같은 페이지에서 탭을 끄지 않고 클릭/스크롤만으로 다음 목표를 찾는지 확인합니다.
config_v3 = {"configurable": {"thread_id": "test_session_keep_alive"}}

# 1턴: 검색 지시
await web_assistant_agent_v3.ainvoke(
    {"messages": [HumanMessage("위키백과에서 '인공지능'을 검색해서 개요 부분을 요약해줘.")]},
    config=config_v3
)

print("-" * 50)
# 2턴: 맥락 질문 (브라우저가 꺼지지 않고 현재 위키백과 페이지 안에서 이어서 탐색하는지 확인)
response = await web_assistant_agent_v3.ainvoke(
    {"messages": [HumanMessage("거기서 역사 섹션의 첫 문단만 그대로 읽어줄래?")]},
    config=config_v3
)
print(f"\n🤖 Assistant: {response['messages'][-1].content}")

# ⚠️ 실습이 끝난 후에는 수동으로 브라우저를 반드시 종료시켜주어야 합니다.
await shared_browser.stop()
print("\n🛑 공유 브라우저 세션 종료 완료")

### 실습: Navigator 앱 서버 연동하기

이제 우리가 만든 훌륭한 '웹 브라우징 어시스턴트'를 Jupyter 노트북을 벗어나 실제 서비스처럼 띄워봅시다!
미리 만들어둔 채팅 서빙용 앱(`app`)에 **Navigator 에이전트**를 추가합니다.

이 앱은 README.md에 설명된 것처럼 **FastAPI(백엔드 서버)**와 **Streamlit(프론트엔드 UI)**로 구성된 구조입니다.
우리가 만든 Navigator를 이 서버의 새로운 에이전트로 등록하기 위해 다음 세 가지 단계를 거칩니다.

#### 1. 에이전트 도구(Tool) 등록하기
`app` 프로젝트는 에이전트가 사용하는 도구 목록을 [app/tools]에서 관리하고 있습니다. 

새로운 파일을 만들고(예를 들어, `app/tools/browser_user_tool.py`), 브라우저 탐색 도구를 여기에 먼저 추가합시다.

[app/tools/__init__.py] 파일을 열고, 우리의 도구를 임포트해서 네비게이터 에이전트가 사용할 도구 리스트를 만들어봅시다.

#### 2. 에이전트 생성 스크립트 작성 (app/agents/navigator_agent.py)

(app/agents/navigator_agent.py)
app/agents 폴더 아래에 navigator_agent.py 파일을 새로 생성하고, 방금 등록한 도구들(tools_navigator)을 불러와서 에이전트를 조립하는 파일입니다.

#### 3. 서버 라우터에 등록 (app/server.py)

이제 새로 만든 에이전트를 FastAPI 백엔드 서버에 연결합시다. 

app/agents에 추가 시 자동으로 서버 라우터 등록됩니다. 

app/server.py로 서버를 실행합니다.

🎉 4. 실행 및 테스트

README.md의 Usage Guide에 나온 방식과 동일하게, 프로젝트 루트 디렉토리(AAWS_project/)에서 서버와 UI를 실행합니다.


1번 터미널: FastAPI 백엔드 서버 실행
`python app/server.py`


2번 터미널:

- CLI 테스트 툴로 빠른 확인
`python app/client.py`

- 또는 streamlit 프론트엔드 UI 실행
`streamlit run app/ui.py`


CLI 테스트: 혹시 브라우저 챗봇 대신 터미널에서 테스트하려면 python app/client.py를 실행하고 /switch navigator_agent 명령어를 쳐서 에이전트를 변경해 보세요!


브라우저 창의 챗봇(Streamlit UI)에서 Agent를 navigator_agent로 바꾸고 대화해 보세요. 질문을 던지면 vnc 화면에서 브라우저가 자동으로 움직이며 맥락을 유지한 상태로 검색을 수행하는 것을 볼 수 있습니다!